# PyTorch Deep Dive: Image Classification

Regression is cool, but AI is famous for **Vision**. 

In this notebook, we will teach a computer to "see" by classifying images (FashionMNIST). 

## Learning Objectives
- **The Vocabulary**: What is a "Pixel", "Channel", "Class", "Logits", and "Softmax"?
- **The Intuition**: How a computer sees an image (Grid of numbers).
- **The Practice**: Building a classifier for 10 types of clothing.
- **The Visual**: Seeing the model's predictions.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

torch.manual_seed(42)

## Part 1: The Vocabulary (Definitions First)

Computer Vision has its own language. Let's learn it.

### 1. Pixel
- The smallest dot in an image.
- Represented as a number from 0 (Black) to 255 (White).
- In PyTorch, we usually normalize this to be between 0 and 1.

### 2. Channel
- The color layers of an image.
- **Grayscale**: 1 Channel (Brightness).
- **Color (RGB)**: 3 Channels (Red, Green, Blue).

### 3. Class
- The category we want to predict.
- Example: "T-Shirt", "Trouser", "Sneaker".
- We map these to numbers: 0, 1, 2...

### 4. Logits
- The raw, unnormalized scores coming out of the last layer of the model.
- They can be any number (e.g., -5.2, 12.8).
- Higher score = Higher confidence.

### 5. Softmax
- A function that turns Logits into Probabilities (0 to 1).
- It makes sure all probabilities add up to 100%.

## Part 2: The Intuition (The Grid)

To you, an image is a picture of a shoe.
To a computer, an image is just a **Grid of Numbers**.

If you have a 28x28 image, you have 784 numbers.
The model's job is to look at these 784 numbers and say "That pattern looks like a Shoe".

In [ ]:
import numpy as np

# Create a simple demonstration
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Get a sample image from FashionMNIST
sample_img = images[0].squeeze().numpy()

# 1. Original image
axes[0, 0].imshow(sample_img, cmap='gray')
axes[0, 0].set_title('What YOU See\n(A Shoe)', fontsize=12, fontweight='bold')
axes[0, 0].axis('off')

# 2. Pixel values as heatmap
im = axes[0, 1].imshow(sample_img, cmap='viridis')
axes[0, 1].set_title('What COMPUTER Sees\n(Grid of Numbers)', fontsize=12, fontweight='bold')
axes[0, 1].axis('off')
plt.colorbar(im, ax=axes[0, 1], fraction=0.046, pad=0.04)

# 3. Zoomed section showing actual numbers
zoom_size = 8
axes[0, 2].imshow(sample_img[:zoom_size, :zoom_size], cmap='gray')
axes[0, 2].set_title(f'Zoomed {zoom_size}x{zoom_size} Section', fontsize=12, fontweight='bold')
for i in range(zoom_size):
    for j in range(zoom_size):
        text = axes[0, 2].text(j, i, f'{sample_img[i, j]:.2f}',
                              ha="center", va="center", color="red", fontsize=7, fontweight='bold')
axes[0, 2].set_xticks(range(zoom_size))
axes[0, 2].set_yticks(range(zoom_size))
axes[0, 2].grid(True, color='yellow', linewidth=1.5)

# 4. 1D representation (flattened)
axes[1, 0].plot(sample_img.flatten()[:200], linewidth=1)
axes[1, 0].set_xlabel('Pixel Index', fontsize=11)
axes[1, 0].set_ylabel('Pixel Value', fontsize=11)
axes[1, 0].set_title('Flattened to 1D Vector\n(First 200 pixels)', fontsize=12, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# 5. Histogram of pixel values
axes[1, 1].hist(sample_img.flatten(), bins=50, color='steelblue', edgecolor='black', alpha=0.7)
axes[1, 1].set_xlabel('Pixel Value', fontsize=11)
axes[1, 1].set_ylabel('Frequency', fontsize=11)
axes[1, 1].set_title('Distribution of Pixel Values', fontsize=12, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3, axis='y')

# 6. Shape information
axes[1, 2].axis('off')
info_text = f"""
Image Properties:
━━━━━━━━━━━━━━━━━━━━
Shape:     {sample_img.shape}
Total Pixels: {sample_img.size}
Data Type:    {sample_img.dtype}
Value Range:  [{sample_img.min():.2f}, {sample_img.max():.2f}]

Representation:
• 2D Array of numbers
• Each number = brightness
• 28 rows × 28 columns
• Total: 784 input features

For the neural network:
Input vector has 784 values!
"""
axes[1, 2].text(0.1, 0.5, info_text, fontsize=10, family='monospace',
               bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8),
               verticalalignment='center')

plt.tight_layout()
plt.show()

print("Key Insight:")
print("• Humans see: shapes, patterns, objects")
print("• Computers see: 784 numbers between 0 and 1")
print("• Neural networks learn to map these numbers to labels!")

### Visualization: How Computers See Images

Let's visualize what an image looks like to a computer - just numbers!

## Part 3: The Data (FashionMNIST)

We will use FashionMNIST. It's like "Hello World" for vision, but harder than digits.
- Images: 28x28 Grayscale.
- Classes: 10 (T-shirt, Trouser, Pullover, Dress, Coat, Sandal, Shirt, Sneaker, Bag, Boot).

In [ ]:
# Download Data
transform = transforms.Compose([transforms.ToTensor()])

trainset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# Visualize one batch
dataiter = iter(trainloader)
images, labels = next(dataiter)

print(f"Image Batch Shape: {images.shape} (Batch, Channel, Height, Width)")
print(f"Label Batch Shape: {labels.shape}")

# Show first image
plt.imshow(images[0].squeeze(), cmap='gray')
plt.title(f"Label: {labels[0].item()}")
plt.show()

## Part 4: The Model (Flattening)

Since we are using Linear Layers (for now), we need to **Flatten** the 2D image (28x28) into a 1D vector (784).

Imagine taking the image and cutting it into strips, then laying them end-to-end.

In [ ]:
class VisionNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.layer1 = nn.Linear(28 * 28, 128) # 784 -> 128
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(128, 10)      # 128 -> 10 Classes

    def forward(self, x):
        x = self.flatten(x)
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x) # Output Logits
        return x

model = VisionNet()
criterion = nn.CrossEntropyLoss() # Combines Softmax + NLLLoss
optimizer = optim.Adam(model.parameters(), lr=0.001)

## Part 5: Training (The Loop)

Same loop, but now we iterate over the `trainloader`.

In [ ]:
epochs = 5

for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch {epoch}: Loss {running_loss / len(trainloader):.4f}")

print("Training Complete!")

## Part 6: Evaluation (Accuracy)

How many did we get right?

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy: {100 * correct / total}%")

In [ ]:
# Comprehensive visualization of classification results
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Class names for FashionMNIST
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# Get predictions for test set
all_predictions = []
all_labels = []
all_probs = []

model.eval()
with torch.no_grad():
    for images, labels in testloader:
        outputs = model(images)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        _, predicted = torch.max(outputs, 1)
        
        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu.numpy())
        all_probs.extend(probabilities.cpu().numpy())

all_predictions = np.array(all_predictions)
all_labels = np.array(all_labels)
all_probs = np.array(all_probs)

# Create comprehensive dashboard
fig = plt.figure(figsize=(18, 12))
gs = fig.add_gridspec(3, 4, hspace=0.3, wspace=0.3)

# 1. Confusion Matrix
ax1 = fig.add_subplot(gs[0:2, 0:2])
cm = confusion_matrix(all_labels, all_predictions)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, 
            yticklabels=class_names, ax=ax1, cbar_kws={'label': 'Count'})
ax1.set_xlabel('Predicted Label', fontsize=11)
ax1.set_ylabel('True Label', fontsize=11)
ax1.set_title('Confusion Matrix\n(Diagonal = Correct Predictions)', fontsize=12, fontweight='bold')
plt.setp(ax1.get_xticklabels(), rotation=45, ha='right', fontsize=9)
plt.setp(ax1.get_yticklabels(), rotation=0, fontsize=9)

# 2. Per-class accuracy
ax2 = fig.add_subplot(gs[0, 2:])
class_correct = cm.diagonal()
class_total = cm.sum(axis=1)
class_accuracy = class_correct / class_total * 100

colors = ['green' if acc > 80 else 'orange' if acc > 70 else 'red' for acc in class_accuracy]
bars = ax2.barh(range(10), class_accuracy, color=colors, edgecolor='black')
ax2.set_yticks(range(10))
ax2.set_yticklabels(class_names, fontsize=9)
ax2.set_xlabel('Accuracy (%)', fontsize=11)
ax2.set_title('Per-Class Accuracy', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='x')
ax2.axvline(x=80, color='green', linestyle='--', linewidth=1, alpha=0.5)

for i, (bar, acc) in enumerate(zip(bars, class_accuracy)):
    ax2.text(acc + 1, i, f'{acc:.1f}%', va='center', fontsize=9, fontweight='bold')

# 3. Prediction confidence distribution
ax3 = fig.add_subplot(gs[1, 2])
max_probs = all_probs.max(axis=1)
correct_mask = all_predictions == all_labels

ax3.hist(max_probs[correct_mask], bins=50, alpha=0.7, label='Correct', color='green', edgecolor='black')
ax3.hist(max_probs[~correct_mask], bins=50, alpha=0.7, label='Wrong', color='red', edgecolor='black')
ax3.set_xlabel('Confidence', fontsize=11)
ax3.set_ylabel('Count', fontsize=11)
ax3.set_title('Prediction Confidence\n(Higher = More Certain)', fontsize=11, fontweight='bold')
ax3.legend()
ax3.grid(True, alpha=0.3, axis='y')

# 4. Accuracy by confidence
ax4 = fig.add_subplot(gs[1, 3])
confidence_bins = np.linspace(0, 1, 11)
bin_accuracies = []
bin_centers = []

for i in range(len(confidence_bins) - 1):
    mask = (max_probs >= confidence_bins[i]) & (max_probs < confidence_bins[i+1])
    if mask.sum() > 0:
        acc = (all_predictions[mask] == all_labels[mask]).mean() * 100
        bin_accuracies.append(acc)
        bin_centers.append((confidence_bins[i] + confidence_bins[i+1]) / 2)

ax4.plot(bin_centers, bin_accuracies, marker='o', linewidth=2, markersize=8, color='blue')
ax4.plot([0, 1], [0, 100], 'r--', linewidth=1, alpha=0.5, label='Perfect Calibration')
ax4.set_xlabel('Confidence', fontsize=11)
ax4.set_ylabel('Accuracy (%)', fontsize=11)
ax4.set_title('Calibration Curve', fontsize=11, fontweight='bold')
ax4.grid(True, alpha=0.3)
ax4.legend()
ax4.set_xlim(0, 1)
ax4.set_ylim(0, 105)

# 5. Sample predictions (correct)
ax5 = fig.add_subplot(gs[2, 0:2])
ax5.axis('off')

# Get some correct predictions
correct_indices = np.where(correct_mask)[0][:6]
y_pos = 0.9

for idx, test_idx in enumerate(correct_indices):
    col = idx % 6
    
    # Get image
    test_image = testset.data[test_idx].numpy()
    
    # Create small subplot
    left = 0.02 + col * 0.16
    ax_img = fig.add_axes([left, 0.05, 0.12, 0.12])
    ax_img.imshow(test_image, cmap='gray')
    ax_img.axis('off')
    
    true_label = all_labels[test_idx]
    pred_label = all_predictions[test_idx]
    confidence = max_probs[test_idx]
    
    title = f'{class_names[pred_label]}\n{confidence*100:.0f}% ✓'
    ax_img.set_title(title, fontsize=9, color='green', fontweight='bold')

ax5.text(0.5, 0.95, 'Correct Predictions (High Confidence)', ha='center', 
        fontsize=12, fontweight='bold', transform=ax5.transAxes)

# 6. Sample predictions (incorrect)
ax6 = fig.add_subplot(gs[2, 2:])
ax6.axis('off')

# Get some incorrect predictions
incorrect_indices = np.where(~correct_mask)[0][:6]

for idx, test_idx in enumerate(incorrect_indices):
    col = idx % 6
    
    # Get image
    test_image = testset.data[test_idx].numpy()
    
    # Create small subplot
    left = 0.52 + col * 0.08
    ax_img = fig.add_axes([left, 0.05, 0.06, 0.06])
    ax_img.imshow(test_image, cmap='gray')
    ax_img.axis('off')
    
    true_label = all_labels[test_idx]
    pred_label = all_predictions[test_idx]
    confidence = max_probs[test_idx]
    
    title = f'Pred: {class_names[pred_label][:8]}\nTrue: {class_names[true_label][:8]}'
    ax_img.set_title(title, fontsize=7, color='red', fontweight='bold')

ax6.text(0.5, 0.95, 'Incorrect Predictions (Model Mistakes)', ha='center', 
        fontsize=12, fontweight='bold', transform=ax6.transAxes)

plt.suptitle(f'Classification Dashboard - Overall Accuracy: {correct / total * 100:.2f}%', 
            fontsize=16, fontweight='bold', y=0.98)
plt.show()

print("\nKey Insights:")
print(f"• Overall Accuracy: {correct / total * 100:.2f}%")
print(f"• Best class: {class_names[class_accuracy.argmax()]} ({class_accuracy.max():.1f}%)")
print(f"• Worst class: {class_names[class_accuracy.argmin()]} ({class_accuracy.min():.1f}%)")
print(f"• Average confidence (correct): {max_probs[correct_mask].mean()*100:.1f}%")
print(f"• Average confidence (wrong): {max_probs[~correct_mask].mean()*100:.1f}%")

### Visualization: Model Predictions Analysis

Let's create a comprehensive dashboard to analyze our model's performance.

## Summary of Part 1

1. **Pixel** = Input number (0-1).
2. **Flatten** = Turning a 2D grid into a 1D vector.
3. **Logits** = Raw scores from the model.
4. **CrossEntropyLoss** = The standard loss for classification.

**But wait!** Flattening images throws away spatial structure. This is where **Convolutional Neural Networks (CNNs)** come in.

---

# PART 2: MODERN COMPUTER VISION (FAANG-Level)

Now let's learn the architectures that power Google Photos, Tesla Autopilot, and Meta's image recognition.

## Part 7: Convolutional Neural Networks (CNNs)

### The Problem with Flattening
When we flatten a 28x28 image into 784 numbers, we lose **spatial structure**. A pixel at position (5,5) is neighbors with (5,6) and (6,5), but after flattening, this relationship is destroyed.

### The Solution: Convolution
A **Convolutional Layer** slides a small filter (e.g., 3x3) across the image, preserving spatial relationships.

Think of it like a "pattern detector":
- One filter might detect **horizontal edges**.
- Another might detect **circles**.
- Another might detect **textures**.

### The Math (Simplified)
For each position in the image, multiply the 3x3 filter with the 3x3 patch of pixels, sum them up, and put the result in the output.

$$ Output[i,j] = \sum_{m,n} Input[i+m, j+n] \times Filter[m, n] $$

In [ ]:
# Simple CNN for FashionMNIST
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        # Convolutional Layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)  # Reduces size by half
        
        # Fully Connected Layers
        self.fc1 = nn.Linear(64 * 7 * 7, 128)  # After 2 poolings: 28->14->7
        self.fc2 = nn.Linear(128, 10)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        # Conv Block 1
        x = self.pool(self.relu(self.conv1(x)))  # 28x28 -> 14x14
        # Conv Block 2
        x = self.pool(self.relu(self.conv2(x)))  # 14x14 -> 7x7
        # Flatten
        x = x.view(-1, 64 * 7 * 7)
        # FC Layers
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

cnn_model = SimpleCNN()
print(cnn_model)

# Count parameters
total_params = sum(p.numel() for p in cnn_model.parameters())
print(f"\nTotal parameters: {total_params:,}")

In [ ]:
# Train the CNN (faster and more accurate than the flattened model)
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

cnn_model = cnn_model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)

epochs = 5
for epoch in range(epochs):
    cnn_model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for i, (images, labels) in enumerate(trainloader):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = cnn_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    train_acc = 100 * correct / total
    print(f"Epoch {epoch+1}: Loss {running_loss/len(trainloader):.4f}, Train Acc: {train_acc:.2f}%")

# Test the CNN
cnn_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = cnn_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"\nCNN Test Accuracy: {100 * correct / total:.2f}%")
print("Compare this to the flattened model - CNNs are much better!")

---

## Part 8: ResNet - Residual Networks (FAANG Interview Favorite)

### The Problem: Vanishing Gradients
When you stack many layers (50, 100, 200), gradients become tiny during backprop. Deep networks fail to train.

### The Breakthrough: Skip Connections (2015 - Microsoft Research)
Instead of learning $H(x)$, learn the **residual** $F(x) = H(x) - x$.

$$ y = F(x) + x $$

The "+x" is a **skip connection** (also called shortcut or residual connection).

**Why it works:**
- If the layer should do nothing (identity mapping), it just learns F(x) = 0.
- Gradients can flow directly through the skip connection.
- Won ImageNet 2015 with 152 layers (previous winners had ~20 layers).

### FAANG Interview Question
**"Implement a ResNet block from scratch."** ← This is asked at Google, Meta, Nvidia!

In [ ]:
class ResidualBlock(nn.Module):
    """
    The basic building block of ResNet.
    
    This is the MOST IMPORTANT interview question for Computer Vision roles.
    """
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        
        # Main path (the F(x) part)
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                               stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        # Shortcut path (the +x part)
        # If dimensions change, we need a projection
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, 
                         stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
        
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        # Main path
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        
        # Add skip connection (THE KEY INNOVATION!)
        out += self.shortcut(x)  # This is y = F(x) + x
        out = self.relu(out)
        
        return out

# Test the block
block = ResidualBlock(1, 32)
test_input = torch.randn(1, 1, 28, 28)
test_output = block(test_input)
print(f"Input shape: {test_input.shape}")
print(f"Output shape: {test_output.shape}")
print("✓ Residual block implemented correctly!")

In [ ]:
class ResNet(nn.Module):
    """
    A simplified ResNet for FashionMNIST (28x28 images).
    In production (ImageNet), you'd use ResNet18, ResNet50, ResNet101.
    """
    def __init__(self, num_classes=10):
        super().__init__()
        
        # Initial convolution
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU(inplace=True)
        
        # Residual blocks
        self.layer1 = self._make_layer(32, 32, num_blocks=2, stride=1)
        self.layer2 = self._make_layer(32, 64, num_blocks=2, stride=2)
        self.layer3 = self._make_layer(64, 128, num_blocks=2, stride=2)
        
        # Classifier
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(128, num_classes)
    
    def _make_layer(self, in_channels, out_channels, num_blocks, stride):
        layers = []
        # First block might downsample
        layers.append(ResidualBlock(in_channels, out_channels, stride))
        # Remaining blocks
        for _ in range(1, num_blocks):
            layers.append(ResidualBlock(out_channels, out_channels, stride=1))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        # Stem
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        
        # Residual blocks
        x = self.layer1(x)  # 28x28
        x = self.layer2(x)  # 14x14
        x = self.layer3(x)  # 7x7
        
        # Classifier
        x = self.avgpool(x)  # 1x1
        x = torch.flatten(x, 1)
        x = self.fc(x)
        
        return x

# Create ResNet
resnet = ResNet(num_classes=10)
print(resnet)

# Count parameters
total_params = sum(p.numel() for p in resnet.parameters())
print(f"\nTotal parameters: {total_params:,}")

# Test forward pass
test_input = torch.randn(4, 1, 28, 28)
test_output = resnet(test_input)
print(f"\nInput shape: {test_input.shape}")
print(f"Output shape: {test_output.shape}")
print("✓ ResNet works!")

---

## Part 9: Vision Transformers (ViT) - The Future (2021-2025)

### The Paradigm Shift
In 2020, Google Research asked: **"What if we applied Transformers to images?"**

Result: **Vision Transformer (ViT)** beats CNNs on ImageNet (2021).
- Used in DALL-E, Stable Diffusion, Segment Anything (SAM).
- **Standard at Google, Meta, OpenAI in 2025.**

### The Core Idea: Patches
Instead of convolutions, we:
1. Split the image into fixed-size **patches** (e.g., 16x16).
2. Flatten each patch into a vector (like a "word" in NLP).
3. Feed these patches to a Transformer.

For a 224x224 image with 16x16 patches:
- Number of patches = (224/16) × (224/16) = 14 × 14 = 196 patches
- Each patch is a "token" (like a word in a sentence)

### The Architecture
```
Image (224x224x3)
  → Split into patches (196 patches of 16x16x3)
  → Linear projection (embed each patch)
  → Add positional embeddings
  → Transformer Encoder
  → MLP Head (classification)
```

### FAANG Interview Alert
**"Explain how Vision Transformers differ from CNNs"** ← Asked at Google, Meta

In [ ]:
class PatchEmbedding(nn.Module):
    """
    Convert image into patches and embed them.
    """
    def __init__(self, img_size=28, patch_size=7, in_channels=1, embed_dim=64):
        super().__init__()
        self.patch_size = patch_size
        self.num_patches = (img_size // patch_size) ** 2
        
        # Use a Conv2d to extract patches (clever trick!)
        self.proj = nn.Conv2d(in_channels, embed_dim, 
                             kernel_size=patch_size, stride=patch_size)
    
    def forward(self, x):
        # x: (B, C, H, W)
        x = self.proj(x)  # (B, embed_dim, num_patches_h, num_patches_w)
        x = x.flatten(2)  # (B, embed_dim, num_patches)
        x = x.transpose(1, 2)  # (B, num_patches, embed_dim)
        return x

class VisionTransformer(nn.Module):
    """
    Simplified Vision Transformer for FashionMNIST.
    """
    def __init__(self, img_size=28, patch_size=7, in_channels=1, num_classes=10,
                 embed_dim=64, depth=4, num_heads=4, mlp_ratio=2.0):
        super().__init__()
        
        # 1. Patch Embedding
        self.patch_embed = PatchEmbedding(img_size, patch_size, in_channels, embed_dim)
        num_patches = self.patch_embed.num_patches
        
        # 2. Class token (like [CLS] in BERT)
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        
        # 3. Positional Embedding
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, embed_dim))
        
        # 4. Transformer Encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=num_heads,
            dim_feedforward=int(embed_dim * mlp_ratio),
            dropout=0.1,
            activation='gelu',
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=depth)
        
        # 5. Classification Head
        self.norm = nn.LayerNorm(embed_dim)
        self.head = nn.Linear(embed_dim, num_classes)
    
    def forward(self, x):
        B = x.shape[0]
        
        # 1. Patch embedding
        x = self.patch_embed(x)  # (B, num_patches, embed_dim)
        
        # 2. Add class token
        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat([cls_tokens, x], dim=1)  # (B, num_patches+1, embed_dim)
        
        # 3. Add positional embedding
        x = x + self.pos_embed
        
        # 4. Transformer
        x = self.transformer(x)
        
        # 5. Classification from [CLS] token
        x = self.norm(x[:, 0])  # Take the CLS token
        x = self.head(x)
        
        return x

# Create ViT
vit = VisionTransformer(img_size=28, patch_size=7, num_classes=10)
print(vit)

# Count parameters
total_params = sum(p.numel() for p in vit.parameters())
print(f"\nTotal parameters: {total_params:,}")

# Test
test_input = torch.randn(4, 1, 28, 28)
test_output = vit(test_input)
print(f"\nInput shape: {test_input.shape}")
print(f"Output shape: {test_output.shape}")
print("✓ Vision Transformer works!")

---

## Part 10: Transfer Learning (How FAANG Actually Uses Models)

### The Reality Check
At Google/Meta, you **never** train ImageNet from scratch. You:
1. Download a pre-trained model (ResNet50, EfficientNet, ViT)
2. Fine-tune it on your data

This is 100x faster and uses 100x less data.

### The Strategy
**Fine-tuning options:**
1. **Feature Extraction**: Freeze all layers except the last one
2. **Fine-tuning**: Unfreeze some/all layers and train with small LR
3. **Progressive unfreezing**: Unfreeze layers gradually

### FAANG Interview Question
**"When would you freeze layers vs fine-tune them?"**

**Answer:**
- **Freeze** when: Small dataset, similar to ImageNet
- **Fine-tune** when: Large dataset, different from ImageNet

In [ ]:
# Transfer Learning Example using torchvision pretrained models
# Note: For FashionMNIST we'll demonstrate the concept, 
# but in practice you'd use this for RGB images

from torchvision import models

# Option 1: Load pre-trained ResNet18 (trained on ImageNet)
# pretrained_resnet = models.resnet18(pretrained=True)

# For demonstration, let's show how to modify it for grayscale + 10 classes
class TransferResNet(nn.Module):
    def __init__(self, num_classes=10, freeze_layers=True):
        super().__init__()
        # Load pretrained ResNet18
        self.backbone = models.resnet18(weights=None)  # Would use weights=ResNet18_Weights.DEFAULT for real transfer
        
        # Modify first conv for grayscale (1 channel instead of 3)
        self.backbone.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        
        # Freeze backbone if feature extraction mode
        if freeze_layers:
            for param in self.backbone.parameters():
                param.requires_grad = False
        
        # Replace final FC layer for our 10 classes
        num_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Linear(num_features, num_classes)
        
        # Always train the new FC layer
        for param in self.backbone.fc.parameters():
            param.requires_grad = True
    
    def forward(self, x):
        return self.backbone(x)

# Create transfer learning model
transfer_model = TransferResNet(num_classes=10, freeze_layers=True)

# Count trainable vs frozen parameters
trainable = sum(p.numel() for p in transfer_model.parameters() if p.requires_grad)
total = sum(p.numel() for p in transfer_model.parameters())
print(f"Trainable parameters: {trainable:,} / {total:,} ({100*trainable/total:.1f}%)")
print("\\nIn feature extraction mode, we only train the last layer!")
print("This trains 100x faster than training from scratch.")

---

## Part 11: Data Augmentation (Making Models Robust)

### The Problem
Models overfit on small datasets. A dog facing left looks different from a dog facing right (to the model).

### The Solution: Augmentation
Artificially create variations of your data:
- **Geometric**: Rotation, flipping, cropping, scaling
- **Color**: Brightness, contrast, saturation
- **Advanced**: CutOut, MixUp, CutMix, RandAugment

### The Impact
- Can improve accuracy by 5-15%
- Standard practice at all FAANG companies
- Required for winning Kaggle competitions

### FAANG Interview Question
**"How do you prevent overfitting in computer vision?"**

**Answer:** 
1. Data augmentation
2. Dropout / regularization  
3. More data
4. Early stopping
5. Transfer learning

In [ ]:
# Data Augmentation Pipeline (Production-Ready)

# Basic augmentation for training
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),  # Flip 50% of images
    transforms.RandomRotation(degrees=15),    # Rotate up to 15 degrees
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Shift image
    transforms.RandomResizedCrop(28, scale=(0.8, 1.0)),  # Random zoom
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])

# No augmentation for test (important!)
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Create augmented datasets
train_augmented = torchvision.datasets.FashionMNIST(
    root='./data', train=True, download=True, transform=train_transform
)
test_normal = torchvision.datasets.FashionMNIST(
    root='./data', train=False, download=True, transform=test_transform
)

# Visualize augmented images
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
fig.suptitle("Data Augmentation Examples (Same Image, Different Augmentations)")

# Get one image and apply augmentation 10 times
original_image, label = train_augmented.dataset.data[0], train_augmented.dataset.targets[0]

for i in range(10):
    augmented = train_transform(transforms.ToPILImage()(original_image))
    row = i // 5
    col = i % 5
    axes[row, col].imshow(augmented.squeeze(), cmap='gray')
    axes[row, col].axis('off')

plt.tight_layout()
plt.show()

print("✓ Each training image is randomly transformed during training!")
print("This creates infinite variations and prevents overfitting.")

---

## Final Summary: Computer Vision for FAANG Interviews

### Architecture Evolution (What You MUST Know)

| Year | Architecture | Key Innovation | When Used |
|------|--------------|----------------|-----------|
| 2012 | **AlexNet** | First deep CNN to win ImageNet | Legacy systems |
| 2014 | **VGG** | Deeper networks (16-19 layers) | Feature extraction |
| 2015 | **ResNet** ⭐ | Skip connections | **Most interviews** |
| 2016 | **Inception** | Multi-scale convolutions | Google products |
| 2017 | **MobileNet** | Depthwise separable convs | Mobile/edge devices |
| 2019 | **EfficientNet** | Compound scaling | SOTA 2019-2021 |
| 2021 | **ViT** ⭐ | Transformers for images | **Future standard** |
| 2022 | **ConvNeXt** | Modernized CNNs | Research |

⭐ = Must implement from scratch for interviews

### FAANG Interview Preparation Checklist

#### **Must Know (100% Chance of Being Asked)**
- ✅ Explain convolution vs fully connected
- ✅ Implement ResNet block from scratch
- ✅ Explain skip connections and why they work
- ✅ Batch Normalization: what, why, when
- ✅ Difference between CNN and ViT
- ✅ Transfer learning strategy
- ✅ Data augmentation techniques

#### **Should Know (70% Chance)**
- ✅ Depthwise separable convolutions (MobileNet)
- ✅ Global Average Pooling vs Fully Connected
- ✅ Receptive field calculation
- ✅ Spatial Pyramid Pooling
- ✅ Gradient flow in very deep networks

#### **Nice to Know (30% Chance)**
- ✅ Neural Architecture Search (NAS)
- ✅ Knowledge distillation
- ✅ Pruning and quantization
- ✅ Attention mechanisms in CNNs (CBAM, SENet)

### Common Interview Questions (Copy These!)

**Q1: "Why do ResNets work better than plain deep networks?"**
```
Answer:
1. Skip connections allow gradients to flow directly through the network
2. Enables learning identity mappings (if layer should do nothing, learn F(x)=0)
3. Prevents vanishing gradients in very deep networks (50-200 layers)
4. Allows training much deeper networks without degradation
```

**Q2: "When would you use a CNN vs a Vision Transformer?"**
```
Answer:
CNN:
- Small datasets (CNNs have inductive bias)
- Need translation equivariance
- Limited compute
- Edge deployment (faster inference)

ViT:
- Large datasets (100M+ images)
- Pretraining foundation models
- Don't need strict spatial invariance
- Have massive compute for training
```

**Q3: "Implement a 3x3 convolution in Python/NumPy"**
```python
# This is asked at Google/Meta!
def conv2d(input, kernel):
    h, w = input.shape
    kh, kw = kernel.shape
    output = np.zeros((h - kh + 1, w - kw + 1))
    
    for i in range(output.shape[0]):
        for j in range(output.shape[1]):
            output[i, j] = np.sum(input[i:i+kh, j:j+kw] * kernel)
    
    return output
```

### What We Covered in This Enhanced Notebook

1. ✅ **Basic Classification** (MLP on flattened images)
2. ✅ **CNNs** (Spatial structure preservation)
3. ✅ **ResNet** (Skip connections, deep networks)
4. ✅ **Vision Transformers** (Patch-based, attention)
5. ✅ **Transfer Learning** (Fine-tuning pretrained models)
6. ✅ **Data Augmentation** (Preventing overfitting)

### Next Steps for FAANG Prep

1. **Implement from scratch:** ResNet-18, ViT-Tiny
2. **Read papers:** ResNet, ViT, EfficientNet
3. **Practice:** LeetCode-style CV problems
4. **Build project:** Image classifier end-to-end (data → training → deployment)
5. **Study:** Object detection (YOLO, Faster R-CNN)

### Resources
- Papers: [ResNet](https://arxiv.org/abs/1512.03385), [ViT](https://arxiv.org/abs/2010.11929)
- Code: [torchvision.models](https://pytorch.org/vision/stable/models.html)
- Practice: [Deep Learning Interviews Book](https://arxiv.org/abs/2201.00650)

---

**You are now ready for Computer Vision interviews at FAANG/Nvidia! 🚀**

---

# PART 3: BEYOND CLASSIFICATION - OBJECT DETECTION & SEGMENTATION

Classification answers "What?". Detection answers "What and Where?". Segmentation answers "Where exactly, pixel-by-pixel?".

## Part 12: Object Detection (YOLO Conceptual Understanding)

### The Problem
Classification: "This image contains a dog" ✓  
Detection: "This image contains a dog at (x1,y1,x2,y2)" ✓✓

### The Paradigms

**Two-Stage Detectors (R-CNN Family):**
1. Propose regions (where objects might be)
2. Classify each region

**One-Stage Detectors (YOLO, SSD):**
- Directly predict bounding boxes + classes in one pass
- Much faster (real-time!)

### YOLO: You Only Look Once

**Core Idea:** Divide image into grid, each cell predicts bounding boxes.

```
Image → Grid (7x7) → Each cell predicts:
  - Bounding boxes (x, y, w, h)
  - Confidence scores
  - Class probabilities
```

### FAANG Interview Question
**"Explain the difference between one-stage and two-stage object detectors"**

**Answer:**
- **Two-stage** (Faster R-CNN): Propose regions → Classify. Accurate but slow.
- **One-stage** (YOLO): Direct prediction. Fast (real-time) but less accurate on small objects.
- **Modern trend**: One-stage (YOLO-v8, DETR) catching up in accuracy.

In [ ]:
# Simplified Object Detection Head (Conceptual)
# In practice, you'd use torchvision.models.detection

class SimpleDetectionHead(nn.Module):
    """
    Simplified detection head to understand the concept.
    Real YOLO is more complex but follows this pattern.
    """
    def __init__(self, backbone_out_channels=128, num_classes=10, num_anchors=3):
        super().__init__()
        self.num_classes = num_classes
        self.num_anchors = num_anchors
        
        # For each anchor box, predict:
        # - 4 values for bounding box (x, y, w, h)
        # - 1 value for objectness score
        # - num_classes values for class probabilities
        outputs_per_anchor = 4 + 1 + num_classes
        
        self.detection_head = nn.Conv2d(
            backbone_out_channels,
            num_anchors * outputs_per_anchor,
            kernel_size=1
        )
    
    def forward(self, x):
        """
        x: Feature map from backbone (B, C, H, W)
        Returns: Predictions (B, H, W, num_anchors, 4+1+num_classes)
        """
        batch_size = x.size(0)
        predictions = self.detection_head(x)
        
        # Reshape to (B, num_anchors, 4+1+num_classes, H, W)
        predictions = predictions.view(
            batch_size,
            self.num_anchors,
            -1,
            predictions.size(2),
            predictions.size(3)
        )
        
        # Permute to (B, H, W, num_anchors, 4+1+num_classes)
        predictions = predictions.permute(0, 3, 4, 1, 2)
        
        return predictions

# Example usage
backbone_features = torch.randn(2, 128, 7, 7)  # Features from ResNet
detector = SimpleDetectionHead(backbone_out_channels=128, num_classes=10)
detections = detector(backbone_features)

print(f"Input features: {backbone_features.shape}")
print(f"Detection output: {detections.shape}")
print(f"Interpretation: (Batch, Grid_H, Grid_W, Anchors, [x,y,w,h,conf,class_probs])")
print(f"Each grid cell predicts {detections.size(3)} bounding boxes")
print("\\n✓ This is the core idea behind YOLO!")
print("\\nReal implementation: Use torchvision.models.detection.fasterrcnn_resnet50_fpn()")

---

## Part 13: Semantic Segmentation (Pixel-Perfect Predictions)

### The Problem
- Classification: "Dog" ✓
- Detection: "Dog at (100, 100, 300, 300)" ✓✓
- Segmentation: "These exact pixels are dog" ✓✓✓

### Use Cases
- Medical imaging (tumor segmentation)
- Autonomous driving (road, pedestrian, car segmentation)
- Background removal (Zoom virtual backgrounds)

### U-Net Architecture (The Standard)

```
Input Image
    ↓
Encoder (Downsampling)
    ↓
Bottleneck (Lowest resolution, highest channels)
    ↓
Decoder (Upsampling) + Skip Connections
    ↓
Output Mask (same size as input)
```

**Key Innovation:** Skip connections from encoder to decoder
- Encoder captures "what" (semantic info)
- Decoder captures "where" (spatial info)
- Skip connections combine both!

### FAANG Interview Question
**"Explain the U-Net architecture and why it works for segmentation"**

**Answer:**
1. **Symmetric encoder-decoder** structure
2. **Skip connections** preserve spatial information lost during downsampling  
3. **Pixel-wise prediction** - each pixel gets a class label
4. **Works with small datasets** (important for medical imaging)

In [ ]:
# U-Net Implementation (Simplified for Understanding)

class DoubleConv(nn.Module):
    """Conv -> BN -> ReLU -> Conv -> BN -> ReLU"""
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x):
        return self.double_conv(x)

class UNet(nn.Module):
    """
    Simplified U-Net for semantic segmentation.
    This is THE standard architecture for medical imaging and segmentation tasks.
    """
    def __init__(self, in_channels=1, num_classes=2):
        super().__init__()
        
        # Encoder (Downsampling path)
        self.enc1 = DoubleConv(in_channels, 64)
        self.enc2 = DoubleConv(64, 128)
        self.enc3 = DoubleConv(128, 256)
        
        # Bottleneck
        self.bottleneck = DoubleConv(256, 512)
        
        # Decoder (Upsampling path)
        self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec3 = DoubleConv(512, 256)  # 512 because of skip connection
        
        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec2 = DoubleConv(256, 128)
        
        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec1 = DoubleConv(128, 64)
        
        # Final output layer
        self.out = nn.Conv2d(64, num_classes, kernel_size=1)
        
        self.pool = nn.MaxPool2d(2, 2)
    
    def forward(self, x):
        # Encoder
        enc1 = self.enc1(x)          # 64 channels
        enc2 = self.enc2(self.pool(enc1))  # 128 channels
        enc3 = self.enc3(self.pool(enc2))  # 256 channels
        
        # Bottleneck
        bottleneck = self.bottleneck(self.pool(enc3))  # 512 channels
        
        # Decoder with skip connections
        dec3 = self.upconv3(bottleneck)
        dec3 = torch.cat([dec3, enc3], dim=1)  # Skip connection!
        dec3 = self.dec3(dec3)
        
        dec2 = self.upconv2(dec3)
        dec2 = torch.cat([dec2, enc2], dim=1)  # Skip connection!
        dec2 = self.dec2(dec2)
        
        dec1 = self.upconv1(dec2)
        dec1 = torch.cat([dec1, enc1], dim=1)  # Skip connection!
        dec1 = self.dec1(dec1)
        
        # Output
        out = self.out(dec1)
        return out

# Test U-Net
unet = UNet(in_channels=1, num_classes=2)  # Binary segmentation
test_input = torch.randn(1, 1, 64, 64)  # 64x64 image
test_output = unet(test_input)

print(f"Input shape:  {test_input.shape}")
print(f"Output shape: {test_output.shape}")
print(f"\\n✓ Output has same spatial dimensions as input!")
print(f"✓ Each pixel gets a prediction for {test_output.size(1)} classes")
print(f"\\nSkip connections preserve spatial details lost during downsampling.")

# Count parameters
total_params = sum(p.numel() for p in unet.parameters())
print(f"\\nTotal parameters: {total_params:,}")